In [195]:
import pytorch_lightning as pl
import numpy as np
import torchaudio
import torch
import torch.optim
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd

from torch.utils.data import DataLoader
from torch import nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoderLayer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from utils import WaveNet, AudioMELSpectogramDataset, ContentLoss, StyleLoss

In [196]:
class StyleAutoEncoder(nn.Module):

    def __init__(self, in_channels, shared_channels, out_channels, kernel_size, dilation, nhead=8, dropout=0.2, dim_feedforward=1024):
        super(StyleAutoEncoder, self).__init__()
        self.sharedEncoder = WaveNet(in_channels, shared_channels, kernel_size, dilation, dropout)
        self.speechEncoder = WaveNet(shared_channels, out_channels, kernel_size, dilation, dropout)
        self.styleEncoder = WaveNet(shared_channels, out_channels, kernel_size, dilation, dropout)
        self.style_encoder_attention = TransformerEncoderLayer(
            d_model=out_channels,
            nhead=8,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu'
        )
        self.speech_encoder_attention = TransformerEncoderLayer(
            d_model=out_channels,
            nhead=8,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu'
        )
        self.decoder_attention = TransformerDecoderLayer(
            d_model=out_channels*2,
            nhead=8,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu'
        )
        self.decoder = WaveNet(out_channels*2, in_channels, kernel_size, dilation, dropout)

    def forward(self, speech, style):

        # print('speech', speech.shape)
        # print('style', style.shape)

        style = self.sharedEncoder(style)
        speech = self.sharedEncoder(speech) 

        style = self.styleEncoder(style)
        speech = self.speechEncoder(speech)
        
        style = style.permute(2, 0, 1)  # Reordenamos las dimensiones para que sea compatible con la capa de atención
        style_encoder = self.style_encoder_attention(style)
        style = style.permute(1, 2, 0)
        style_encoder = speech.permute(0, 1, 2)

        speech = speech.permute(2, 0, 1)
        speech_encoder = self.speech_encoder_attention(speech)
        speech = speech.permute(1, 2, 0)
        speech_encoder = speech.permute(0, 1, 2)
        
        # print('speech', speech.shape)
        # print('style', style.shape)
        # print('speech_encoder', speech_encoder.shape)
        # print('style_encoder', style_encoder.shape)

        out = torch.cat((speech, style), dim=1)
        out_encoder = torch.cat((speech_encoder, style_encoder), dim=1)

        out = out.permute(2, 0, 1) 
        out_encoder = out.permute(0, 1, 2) 

        # print('out', out.shape)
        # print('out_encoder', out_encoder.shape)

        x = self.decoder_attention(out, out_encoder)
        x = x.permute(1, 2, 0) 

        # print('x', x.shape)

        x = self.decoder(x)
        x = x.unsqueeze(1)
        # print('x', x.shape)

        return x #.squeeze(0)
 
class MelSpecVCAutoencoderModule(pl.LightningModule):

    def __init__(self, in_channels, shared_channels, out_channels, kernel_size, dilation, device, sample_rate, dropout=0.2):
        super(MelSpecVCAutoencoderModule, self).__init__()
        self.encoder_generator = StyleAutoEncoder(in_channels, shared_channels, out_channels, kernel_size, dilation, dropout)
        self.todevice = device

    def forward(self, speech, style):
        y = self.encoder_generator(speech, style)
        return y

    def training_step(self, batch, batch_idx):
        speech, style = batch
        if speech.ndim == 2: speech = speech.unsqueeze(1)
        if style.ndim == 2: style = style.unsqueeze(1)
        y = self(speech, style)
        print('y', y.shape)
        print('speech', speech.shape)
        print('style', style.shape)
        loss_speech_loss_fn = ContentLoss(speech)
        loss_style_loss_fn = StyleLoss(style)
        loss_speech = loss_speech_loss_fn(y)
        loss_style = loss_style_loss_fn(y)
        loss = loss_speech + loss_style
        self.log('train_loss', loss.item())
        return loss 
    
    def validation_step(self, batch, batch_idx):
        speech, style = batch
        if speech.ndim == 2: speech = speech.unsqueeze(1)
        if style.ndim == 2: style = style.unsqueeze(1)
        y = self(speech, style)
        loss_speech_loss_fn = ContentLoss(speech)
        loss_style_loss_fn = StyleLoss(style)
        loss_speech = loss_speech_loss_fn(y)
        loss_style = loss_style_loss_fn(y)
        loss = loss_speech + loss_style
        self.log('val_loss', loss.item())
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

In [197]:
torch.set_float32_matmul_precision('medium')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [198]:
batch_size = 4
sample_rate = 16000
div_ratio = 0.8
n_fft = 800
audio_length = 2
in_channels, shared_channels, out_channels, kernel_size, dilation = 128, 64, 32, 3, 12
metadata = pd.read_csv('metadata.csv')
dataset = AudioMELSpectogramDataset(metadata, device, sample_rate=sample_rate, audio_length=audio_length, n_fft=n_fft)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(len(dataset) * div_ratio), len(dataset) - int(len(dataset) * div_ratio)])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, drop_last=True)
model = MelSpecVCAutoencoderModule(in_channels, shared_channels, out_channels, kernel_size, dilation, device=device, sample_rate=sample_rate)
model = model.to(device)

e:\miniconda3\envs\AI\lib\site-packages\torchaudio\functional\functional.py:576: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [199]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='./checkpoints',
    filename='{epoch}-{val_loss:.2f}-{val_r2:.2f}',
    save_top_k=1,
    monitor='val_loss',
    every_n_epochs =1,
    mode='min'
)

trainer = pl.Trainer(
    accelerator='gpu',
    max_epochs=10,
    benchmark=True,
    # deterministic=True,
    precision=16,
    accumulate_grad_batches=6,
    callbacks=[
        checkpoint_callback,
        pl.callbacks.LearningRateMonitor(logging_interval='step')
    ]
)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [200]:
trainer.fit(model, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type             | Params
-------------------------------------------------------
0 | encoder_generator | StyleAutoEncoder | 2.3 M 
-------------------------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
4.506     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.fit(model, train_loader, val_loader)                                                 │
│   2                                                                                              │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\trainer.py:608 in fit          │
│                                                                                                  │
│    605 │   │   """                                                                               │
│    606 │   │   model = self._maybe_unwrap_optimized(model)                                       │
│    607 │   │   self.strategy._lightning_module = model                                           │
│ ❱  608 │   │   call._call_and_handle_interrupt(                                                  │
│    609 │   │   │   self, self._fit_impl, model, train_dataloaders, val_dataloaders, datamodule,  │
│    610 │   │   )                                                                                 │
│    611                                                                                           │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\call.py:38 in                  │
│ _call_and_handle_interrupt                                                                       │
│                                                                                                  │
│   35 │   │   if trainer.strategy.launcher is not None:                                           │
│   36 │   │   │   return trainer.strategy.launcher.launch(trainer_fn, *args, trainer=trainer,     │
│   37 │   │   else:                                                                               │
│ ❱ 38 │   │   │   return trainer_fn(*args, **kwargs)                                              │
│   39 │                                                                                           │
│   40 │   except _TunerExitException:                                                             │
│   41 │   │   trainer._call_teardown_hook()                                                       │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\trainer.py:650 in _fit_impl    │
│                                                                                                  │
│    647 │   │   │   model_provided=True,                                                          │
│    648 │   │   │   model_connected=self.lightning_module is not None,                            │
│    649 │   │   )                                                                                 │
│ ❱  650 │   │   self._run(model, ckpt_path=self.ckpt_path)                                        │
│    651 │   │                                                                                     │
│    652 │   │   assert self.state.stopped                                                         │
│    653 │   │   self.training = False                                                             │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\trainer.py:1112 in _run        │
│                                                                                                  │
│   1109 │   │                                               

In [201]:
random_speech = train_dataset[120]
mel_spectrogram = random_speech[0]
style_spec = random_speech[1]

# Convert mel spectrogram to linear scale
mel_to_linear = torchaudio.transforms.InverseMelScale(n_stft=800)
linear_spectrogram = mel_to_linear(mel_spectrogram)

# Convert linear spectrogram to waveform
griffin_lim = torchaudio.transforms.GriffinLim()
waveform = griffin_lim(linear_spectrogram)

# Plot mel spectrogram
plt.figure(figsize=(10, 4))
plt.imshow(mel_spectrogram.log2(), cmap='inferno', origin='lower', aspect='auto')

# Set frequency axis ticks
num_ticks = 10
freq_bins = np.linspace(0, mel_spectrogram.shape[0], num_ticks)
hz_ticks = torchaudio.transforms.MelScale().mel_to_hz(torch.tensor(freq_bins))
plt.yticks(freq_bins, ["{:.0f}".format(hz) for hz in hz_ticks])

plt.colorbar(format='%+2.0f dB')
plt.xlabel('Time')
plt.ylabel('Frequency (Hz)')
plt.title('Mel Spectrogram')
plt.show()

# display(ipd.Audio(waveform.cpu().numpy(), rate=sample_rate))

plt.show()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│    4                                                                                             │
│    5 # Convert mel spectrogram to linear scale                                                   │
│    6 mel_to_linear = torchaudio.transforms.InverseMelScale(n_stft=800)                           │
│ ❱  7 linear_spectrogram = mel_to_linear(mel_spectrogram)                                         │
│    8                                                                                             │
│    9 # Convert linear spectrogram to waveform                                                    │
│   10 griffin_lim = torchaudio.transforms.GriffinLim()                                            │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\torch\nn\modules\module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\torchaudio\transforms\_transforms.py:515 in forward      │
│                                                                                                  │
│    512 │   │   loss = float("inf")                                                               │
│    513 │   │   for _ in range(self.max_iter):                                                    │
│    514 │   │   │   optim.zero_grad()                                                             │
│ ❱  515 │   │   │   diff = melspec - specgram.matmul(self.fb)                                     │
│    516 │   │   │   new_loss = diff.pow(2).sum(axis=-1).mean()                                    │
│    517 │   │   │   # take sum over mel-frequency then average over other dimensions              │
│    518 │   │   │   # so that loss threshold is applied par unit timeframe                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
model.eval()
model = model.to(device)

with torch.no_grad():
    predicted_audio = model.encoder_generator(random_audio.unsqueeze(0), style_audio.unsqueeze(0))

print(random_audio.shape, style_audio.shape, predicted_audio.shape)

display(ipd.Audio(random_audio.cpu().numpy(), rate=sample_rate))
display(ipd.Audio(style_audio.cpu().numpy(), rate=sample_rate))
predicted_audio = predicted_audio.squeeze(0)
display(ipd.Audio(predicted_audio.cpu().numpy(), rate=sample_rate))


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2 model = model.to(device)                                                                    │
│    3                                                                                             │
│    4 with torch.no_grad():                                                                       │
│ ❱  5 │   predicted_audio = model.encoder_generator(random_audio.unsqueeze(0), style_audio.uns    │
│    6                                                                                             │
│    7 print(random_audio.shape, style_audio.shape, predicted_audio.shape)                         │
│    8                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'random_audio' is not defined

In [ ]:
random_speech = train_dataset[1]
mel_spectrogram = random_speech[0]
style_spec = random_speech[1]

mel_spectrogram = mel_spectrogram.unsqueeze(0)
style_spec = style_spec.unsqueeze(0)

print('MEL', mel_spectrogram.shape)
print('style_spec', style_spec.shape)

sharedEncoder = WaveNet(128, out_channels, kernel_size, dilation)

mel_spectrogram = sharedEncoder(mel_spectrogram)
style_spec = sharedEncoder(style_spec)

print('MEL', mel_spectrogram.shape)

# spec image to encoder transformer layer
encoder = TransformerEncoderLayer(
    d_model=64,
    nhead=8,
    dim_feedforward=2048,
    dropout=0.2,
    activation='relu'
)

mel_spectrogram = mel_spectrogram.permute(2,0,1)
print('MEL', mel_spectrogram.shape)

x = encoder(mel_spectrogram)
x = x.permute(1,2,0)
mel_spectrogram = mel_spectrogram.permute(1,2,0)

mel_spectrogram += x

print('x', x.shape)
print('MEL', mel_spectrogram.shape)
print('style_spec', style_spec.shape)

mel_spectrogram = mel_spectrogram.permute(2,0,1)
style_spec = style_spec.permute(2,0,1)
decoder_attention = TransformerDecoderLayer(
    d_model=64,
    nhead=8,
    dim_feedforward=2048,
    dropout=0.2,
    activation='relu'
)

x = decoder_attention(style_spec, mel_spectrogram)

print('x', x.shape)
print('MEL', mel_spectrogram.shape)
print('style_spec', style_spec.shape)

MEL torch.Size([1, 128, 161])
style_spec torch.Size([1, 128, 161])
MEL torch.Size([1, 32, 161])
MEL torch.Size([161, 1, 32])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:30                                                                                   │
│                                                                                                  │
│   27 mel_spectrogram = mel_spectrogram.permute(2,0,1)                                            │
│   28 print('MEL', mel_spectrogram.shape)                                                         │
│   29                                                                                             │
│ ❱ 30 x = encoder(mel_spectrogram)                                                                │
│   31 x = x.permute(1,2,0)                                                                        │
│   32 mel_spectrogram = mel_spectrogram.permute(1,2,0)                                            │
│   33                                                                                             │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\torch\nn\modules\module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\torch\nn\modules\transformer.py:573 in forward           │
│                                                                                                  │
│   570 │   │   │   x = x + self._sa_block(self.norm1(x), src_mask, src_key_padding_mask)          │
│   571 │   │   │   x = x + self._ff_block(self.norm2(x))                                          │
│   572 │   │   else:                                                                              │
│ ❱ 573 │   │   │   x = self.norm1(x + self._sa_block(x, src_mask, src_key_padding_mask))          │
│   574 │   │   │   x = self.norm2(x + self._ff_block(x))                                          │
│   575 │   │                                                                                      │
│   576 │   │   return x                                                                           │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\torch\nn\modules\transformer.py:581 in _sa_block         │
│                                                                                                  │
│   578 │   # self-attention block                                                                 │
│   579 │   def _sa_block(self, x: Tensor,                                                         │
│   580 │   │   │   │     attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor]) -> Te   │
│ ❱ 581 │   │   x = self.self_attn(x, x, x,                                                        │
│   582 │   │   │   │   │   │      attn_mask=attn_mask,                                            │
│   583 │   │   │   │   │   │      key_padding_mask=key_padding_mask,                              │
│   584 │   │   │   │   │   │      need_weights=False)[0]    